In [2]:
# %%
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
from datetime import datetime
import plotly.express as px
import plotly.io as pio

pio.renderers.default = "notebook_connected"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

BASE_URL = "https://www12.senado.leg.br/noticias/ultimas"

print("Scraper Senado Notícias pronto!")


Scraper Senado Notícias pronto!


In [3]:
# %%
def montar_url(pagina):
    if pagina == 1:
        return BASE_URL
    return f"{BASE_URL}/{pagina}"


In [4]:
# %%
noticias = []
TOTAL_PAGES = 5

for pagina in range(TOTAL_PAGES, 0, -1):
    url = montar_url(pagina)
    print(f"Coletando página {pagina}: {url}")

    r = requests.get(url, headers=HEADERS, timeout=15)
    if r.status_code != 200:
        print("  Erro ao acessar página")
        continue

    soup = BeautifulSoup(r.text, "html.parser")

    lista = soup.find("ol", class_="lista-resultados")
    if not lista:
        print("  Nenhuma lista encontrada")
        continue

    itens = lista.find_all("li", recursive=False)
    print(f"  {len(itens)} notícias encontradas")

    for item in itens:
        titulo_tag = item.find("span", class_="eta")
        link_tag = item.find("a", href=True)
        data_tag = item.select_one("div.text-muted.normalis")

        img_tag = item.find("img")

        if not titulo_tag or not link_tag:
            continue

        link = link_tag["href"]
        if link.startswith("/"):
            link = "https://www12.senado.leg.br" + link

        noticias.append({
            "titulo": titulo_tag.get_text(strip=True),
            "data": data_tag.get_text(strip=True) if data_tag else "NA",
            "link": link,
            "imagem": img_tag["src"] if img_tag else "NA",
            "pagina": pagina,
            "fonte": "Agência Senado"
        })

    time.sleep(1)

print(f"\nTotal coletado: {len(noticias)} notícias")


Coletando página 5: https://www12.senado.leg.br/noticias/ultimas/5
  20 notícias encontradas
Coletando página 4: https://www12.senado.leg.br/noticias/ultimas/4
  20 notícias encontradas
Coletando página 3: https://www12.senado.leg.br/noticias/ultimas/3
  20 notícias encontradas
Coletando página 2: https://www12.senado.leg.br/noticias/ultimas/2
  20 notícias encontradas
Coletando página 1: https://www12.senado.leg.br/noticias/ultimas
  20 notícias encontradas

Total coletado: 98 notícias


In [5]:
# %%
df_senado = pd.DataFrame(noticias)
df_senado.head(10)


,titulo,data,link,imagem,pagina,fonte
0,Criação do Dia da Lei Seca em 19 de junho vai ...,11/12/2025 14h10,https://www12.senado.leg.br/noticias/audios/20...,NA,5,Agência Senado
1,CCJ aprova Marco Legal da Cibersegurança,11/12/2025 14h06,https://www12.senado.leg.br/noticias/audios/20...,NA,5,Agência Senado
2,‘PL da Dosimetria’: Esperidião Amin quer entre...,11/12/2025 12h52,https://www12.senado.leg.br/noticias/videos/20...,/noticias/videos/2025/12/2018pl-da-dosimetria2...,5,Agência Senado
3,Congresso deve votar Orçamento de 2026 na próx...,11/12/2025 11h11,https://www12.senado.leg.br/noticias/materias/...,/noticias/materias/2025/12/11/congresso-deve-v...,5,Agência Senado
4,Transferência de profissionais ao Mais Médicos...,11/12/2025 10h06,https://www12.senado.leg.br/noticias/videos/20...,/noticias/videos/2025/12/transferencia-de-prof...,5,Agência Senado
5,Debatedores questionam regras para migração de...,11/12/2025 10h05,https://www12.senado.leg.br/noticias/audios/20...,/noticias/audios/2025/12/debatedores-questiona...,5,Agência Senado
6,PL Antifacção: relator Alessandro Vieira expli...,11/12/2025 09h59,https://www12.senado.leg.br/noticias/audios/20...,/noticias/audios/2025/12/pl-antifaccao-relator...,5,Agência Senado
7,Dois projetos sobre direitos das pessoas autis...,11/12/2025 09h49,https://www12.senado.leg.br/noticias/videos/20...,/noticias/videos/2025/12/dois-projetos-sobre-d...,5,Agência Senado
8,Plínio critica proposta de nova Lei de Impeach...,11/12/2025 09h09,https://www12.senado.leg.br/noticias/materias/...,/noticias/materias/2025/12/11/plinio-critica-p...,5,Agência Senado
9,PEC da polícia científica conclui primeiro tur...,10/12/2025 21h34,https://www12.senado.leg.br/noticias/materias/...,/noticias/materias/2025/12/10/pec-da-policia-c...,5,Agência Senado


In [6]:
# %%
keywords = [
    "agro", "agricultura", "pecuária", "rural",
    "meio ambiente", "clima", "energia", "infraestrutura"
]

pattern = r"|".join(keywords)

df_filt = df_senado[
    df_senado["titulo"].str.contains(pattern, case=False, na=False, regex=True)
].copy()

print(f"{len(df_filt)} notícias filtradas (de {len(df_senado)})")
df_filt.head()


3 notícias filtradas (de 98)


,titulo,data,link,imagem,pagina,fonte
25,Novo prazo para regularização de imóvel rural ...,11/12/2025 17h39,https://www12.senado.leg.br/noticias/materias/...,/noticias/materias/2025/12/11/novo-prazo-para-...,4,Agência Senado
64,Avança projeto que susta norma sobre prorrogaç...,16/12/2025 12h47,https://www12.senado.leg.br/noticias/materias/...,/noticias/materias/2025/12/16/avanca-projeto-q...,2,Agência Senado
95,Suspensão de decreto federal sobre concessões ...,16/12/2025 14h28,https://www12.senado.leg.br/noticias/audios/20...,NA,1,Agência Senado


In [7]:
# %%
def plot_charts(df):
    if df.empty:
        print("Sem dados para gráficos")
        return

    # Top 15 notícias
    top15 = df.head(15).copy()
    top15["rank"] = range(1, len(top15) + 1)

    fig1 = px.bar(
        top15,
        x="rank",
        y="titulo",
        orientation="h",
        title="Top 15 Notícias – Senado (Filtro Agro)"
    )
    fig1.update_layout(height=600)
    fig1.show()

    # Nuvem de palavras
    text = " ".join(df["titulo"]).lower()
    words = re.findall(r"\b\w{4,}\b", text)

    if words:
        wc = pd.Series(words).value_counts().head(20).reset_index()
        wc.columns = ["palavra", "freq"]

        fig2 = px.treemap(
            wc,
            path=["palavra"],
            values="freq",
            title="Nuvem de Palavras – Senado"
        )
        fig2.show()

plot_charts(df_filt if not df_filt.empty else df_senado)
